# Sample for KFServing SDK 

This is a sample for KFServing SDK. 

The notebook shows how to use KFServing SDK to create, get, rollout_canary, promote and delete KFService.

In [ ]:
from kubernetes import client

from kfserving import KFServingClient
from kfserving import constants
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2KFServiceSpec
from kfserving import V1alpha2KFService
from kubernetes.client import V1ResourceRequirements

## Define KFService

Firstly define default endpoint spec, and then define the kfservice basic on the endpoint spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                            tensorflow=V1alpha2TensorflowSpec(
                              storage_uri='gs://kfserving-samples/models/tensorflow/flowers',
                              resources=V1ResourceRequirements(
                                  requests={'cpu':'100m','memory':'1Gi'},
                                  limits={'cpu':'100m', 'memory':'1Gi'}))))
    
kfsvc = V1alpha2KFService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='flower-sample', namespace='kubeflow'),
                          spec=V1alpha2KFServiceSpec(default=default_endpoint_spec))

## Create KFService

Call KFServingClient to create KFService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(kfsvc)

## Check the KFService

In [ ]:
KFServing.get('flower-sample', namespace='kubeflow', watch=True, timeout_seconds=120)

## Add Canary to KFService

Firstly define canary endpoint spec, and then rollout 10% traffic to the canary version, watch the rollout process.

In [ ]:
canary_endpoint_spec = V1alpha2EndpointSpec(
                         predictor=V1alpha2PredictorSpec(
                           tensorflow=V1alpha2TensorflowSpec(
                             storage_uri='gs://kfserving-samples/models/tensorflow/flowers-2',
                             resources=V1ResourceRequirements(
                                 requests={'cpu':'100m','memory':'1Gi'},
                                 limits={'cpu':'100m', 'memory':'1Gi'}))))

KFServing.rollout_canary('flower-sample', canary=canary_endpoint_spec, percent=10,
                         namespace='kubeflow', watch=True, timeout_seconds=120)

## Rollout more traffic to canary of the KFService

Rollout traffice percent to 50% to canary version.

In [ ]:
KFServing.rollout_canary('flower-sample', percent=50, namespace='kubeflow',
                         watch=True, timeout_seconds=120)

## Promote Canary to Default

In [ ]:
KFServing.promote('flower-sample', namespace='kubeflow', watch=True, timeout_seconds=120)

## Delete the KFService

In [ ]:
KFServing.delete('flower-sample', namespace='kubeflow')